In [1]:
# url for image
img_url = 'https://postfiles.pstatic.net/MjAxOTA3MDdfMjU4/MDAxNTYyNTAxODM2NTIw.VRpcxIr6Mcn_yyiNgmy17gwThBm6mTDbM7WdwzsmK5Eg.94fshQcch7MHY0TCUpMnwAXS0dsIMT6EmslaGAA1E58g.JPEG.pola0216/%EC%84%9C%EA%B0%95%EC%A4%80%EB%82%A8%EC%B9%9C%EC%A7%A416.jpg?type=w966'

# model dir (Directory)
model_dir = 'shape_predictor_68_face_landmarks.dat'

!pip install python-pptx

# import the necessary packages
from collections import OrderedDict
import numpy as np
import cv2
import dlib
import imutils
from google.colab.patches import cv2_imshow

# packages for image retrieval
from urllib import request
from PIL import Image
from io import BytesIO

     |████████████████████████████████| 8.9MB 7.4MB/s 
     |████████████████████████████████| 153kB 45.2MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.18-cp36-none-any.whl size=275707 sha256=f5d38118292da2f8109d9043b80b6720c6290dbff515d5060fbd61ad98f885d7
  Stored in directory: /root/.cache/pip/wheels/1f/1f/2c/29acca422b420a0b5210bd2cd7e9669804520d602d2462f20b
Successfully built python-pptx


In [2]:
facial_features_cordinates = {}

# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
FACIAL_LANDMARKS_INDEXES = OrderedDict([
    ("Mouth", (48, 68)),
    ("Right_Eyebrow", (17, 22)),
    ("Left_Eyebrow", (22, 27)),
    ("Right_Eye", (36, 42)),
    ("Left_Eye", (42, 48)),
    ("Nose", (27, 35)),
    ("Jaw", (0, 17))
])
                                                                                                                                                                                                    

def shape_to_numpy_array(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coordinates = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coordinates[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coordinates


def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):
    # create two copies of the input image -- one for the
    # overlay and one for the final output image
    overlay = image.copy()
    output = image.copy()

    # if the colors list is None, initialize it with a unique
    # color for each facial landmark region
    if colors is None:
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
                  (168, 100, 168), (158, 163, 32),
                  (163, 38, 32), (180, 42, 220)]

    # loop over the facial landmark regions individually
    for (i, name) in enumerate(FACIAL_LANDMARKS_INDEXES.keys()):
        # grab the (x, y)-coordinates associated with the
        # face landmark
        (j, k) = FACIAL_LANDMARKS_INDEXES[name]
        pts = shape[j:k]
        facial_features_cordinates[name] = pts

        # check if are supposed to draw the jawline
        if name == "Jaw":
            # since the jawline is a non-enclosed facial region,
            # just draw lines between the (x, y)-coordinates
            for l in range(1, len(pts)):
                ptA = tuple(pts[l - 1])
                ptB = tuple(pts[l])
                cv2.line(overlay, ptA, ptB, colors[i], 2)

        # otherwise, compute the convex hull of the facial
        # landmark coordinates points and display it
        else:
            hull = cv2.convexHull(pts)
            cv2.drawContours(overlay, [hull], -1, colors[i], -1)

    # apply the transparent overlay
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    # return the output image
    # print(facial_features_cordinates)
    return output


# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(model_dir)

# load the input image, resize it, and convert it to grayscale
# image = cv2.imread('/images/image_1.jpg')
res = request.urlopen(img_url).read()
pil_image = Image.open(BytesIO(res)).convert('RGB')
image = np.array(pil_image)[:, :, ::-1].copy()

image = imutils.resize(image, width=2000)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect faces in the grayscale image
rects = detector(gray, 1)

# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the landmark (x, y)-coordinates to a NumPy array
    shape = predictor(gray, rect)
    shape = shape_to_numpy_array(shape)

    output = visualize_facial_landmarks(image, shape)

    # cv2.imshow("Image", output)
    # cv2_imshow(output)
    # cv2.waitKey(0)


# =========================================================


def face_shuffle(img, part_list=['Left_Eye', 'Nose', 'Right_Eye', 'Mouth']):
  crop_img_list = []

  for part in part_list:
    tmp_x = facial_features_cordinates[part][:, 0]
    tmp_x_min, tmp_x_max = tmp_x.min(), tmp_x.max()

    tmp_y = facial_features_cordinates[part][:, 1]
    tmp_y_min, tmp_y_max = tmp_y.min(), tmp_y.max()

    if part.endswith('Eye'):
      tmp_y_min = int(tmp_y_min*0.90)
    if part.endswith('Nose'):
      tmp_x_min = int(tmp_x_min*0.95)
      tmp_x_max = int(tmp_x_max*1.1)

    crop_img = image[tmp_y_min:tmp_y_max, tmp_x_min:tmp_x_max]
    crop_img_list.append(crop_img)
  return crop_img_list


# get 4 parts of detected face (Leye, Nose, Reye, Mouth)
# print(facial_features_cordinates.keys())
part_name_list = ['Left_Eye', 'Nose', 'Right_Eye', 'Mouth']
part_img_list = face_shuffle(image, part_list=part_name_list)
# for img in part_img_list:
#   cv2_imshow(img)
#   print()
#   cv2.waitKey(0)


# =========================================================


# save img files
part_filename_list = [i+'.png' for i in part_name_list]
original_filename = 'original.png'

for name, arr in zip(part_filename_list, part_img_list):
  im = Image.fromarray(cv2.cvtColor(arr, cv2.COLOR_BGR2RGB))
  im.save(name, dpi=(5000, 5000))
pil_image.save(original_filename, dpi=(5000, 5000))
!ls

drive	      Mouth.png  original.png	sample_data
Left_Eye.png  Nose.png	 Right_Eye.png


In [3]:
# Generate final pptx file
from pptx import Presentation
from pptx.util import Inches

def make_ppt(out_ppt_name, part_filename_list, original_filename):
    # Presentation()을 일종의 템플릿 객체
    this_prs = Presentation()
    """
    slide_layout[0]는 title, subtitle로 구성된 제목 슬라이드 
    slide_layout[1]는 title, text로 구성된 일반적인 슬라이드 레이아웃
    slide_layout[6]은 빈 슬라이드
    """
    slide_layout = this_prs.slide_layouts[6]
    
    this_slide = this_prs.slides.add_slide(slide_layout)
    shapes = this_slide.shapes
    for idx, name in enumerate(part_filename_list):
      if idx < 3:
        shapes.add_picture(name, Inches(0.5+idx*3), Inches(0.5), height=Inches(2.5))
      else:
        shapes.add_picture(name, Inches(2.5), Inches(4), height=Inches(2.5))
    
    this_slide = this_prs.slides.add_slide(slide_layout)
    this_slide.shapes.add_picture(original_filename, Inches(2.5), Inches(0), height=Inches(7))
    
    this_prs.save(out_ppt_name)
    

    # for title, content, img_file_name in content_lst:
    #     this_slide = this_prs.slides.add_slide(slide_layout)
    #     shapes = this_slide.shapes
    #     shapes.title.text = title
    #     shapes.placeholders[1].text = content
    #     # placeholders는 개별 slide에 있는 모든 개체를 가져온다고 보면 됨. 
    #     #shapes.add_picture(img_stream, left, top, height=height)
    #     #shapes.add_picture(img_file_name, left=Inches(5), top=Inches(10))
    #     shapes.add_picture(img_file_name, Inches(2.5), Inches(3.2))
    #     # 변환하지 않고 숫자로 넘기면 잘 되지 않는다. 
    # this_prs.save(out_ppt_name)


make_ppt(out_ppt_name='face_shuffle.pptx', part_filename_list=part_filename_list, original_filename=original_filename)
!ls

drive		   Left_Eye.png  Nose.png      Right_Eye.png
face_shuffle.pptx  Mouth.png	 original.png  sample_data
